In [9]:
# print P(x|yes) and P(x|no) for each attribute x
# P(x|yes) = count of x in yes / count of yes 
# probability of x given yes
# probability of x given no
# posteriotr probability
# also do normalization
import pandas as pd
import numpy as np

data = pd.read_csv('play_tennis.csv')



In [1]:
# def calculate_prior(data, target):
#     classes = data[target].unique()
#     priors = {}
#     for cls in classes:
#         priors[cls] = len(data[data[target] == cls]) / len(data)
#     return priors

# def calculate_likelihood(data, feature, target):
#     likelihoods = {}
#     classes = data[target].unique()
#     for cls in classes:
#         class_data = data[data[target] == cls]
#         feature_values = class_data[feature].unique()
#         likelihoods[cls] = {}
#         for value in feature_values:
#             likelihoods[cls][value] = len(class_data[class_data[feature] == value]) / len(class_data)
#     return likelihoods

# # Calculate posterior probabilities
# def calculate_posterior(priors, likelihoods, sample):
#     posteriors = {}
#     for cls in priors:
#         posteriors[cls] = priors[cls]
#         for feature in sample:
#             if sample[feature] in likelihoods[cls]:
#                 posteriors[cls] *= likelihoods[cls][sample[feature]]
#             else:
#                 posteriors[cls] *= 0
#     return posteriors

# # Predict the class for a new sample
# def predict(data, sample, target):
#     priors = calculate_prior(data, target)
#     likelihoods = {}
#     for feature in data.columns:
#         if feature != target:
#             likelihoods[feature] = calculate_likelihood(data, feature, target)
#     posteriors = calculate_posterior(priors, likelihoods, sample)
#     return max(posteriors, key=posteriors.get)

# # Example usage
# sample = {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'}
# prediction = predict(data, sample, 'PlayTennis')
# print(f'The prediction for the sample {sample} is: {prediction}')

In [ ]:
import pandas as pd
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = {}
        self.cond_probs = {}

    def fit(self, X, y):
        self.class_probs = self.calculate_class_probs(y)
        print("Prior Probabilities (P(class)):")
        print(self.class_probs)
        print()

        self.cond_probs = self.calculate_conditional_probs(X, y)
        print("Conditional Probabilities (P(x | class)):")
        print(self.cond_probs)
        print()

    def calculate_class_probs(self, y):
        class_counts = y.value_counts()
        total_count = len(y)
        class_probs = class_counts / total_count
        return class_probs.to_dict()

    def calculate_conditional_probs(self, X, y):
        cond_probs = {}
        for column in X.columns:
            cond_probs[column] = {}
            for value in X[column].unique():
                for class_value in y.unique():
                    count_x_given_class = len(X[(X[column] == value) & (y == class_value)])
                    count_class = len(y[y == class_value])
                    prob = count_x_given_class / count_class
                    cond_probs[column][(value, class_value)] = prob
        return cond_probs

    def predict(self, X):
        predictions = []
        for index, row in X.iterrows():
            posteriors = {}
            print(f"Predicting for row {index}: {row.to_dict()}")
            for class_value, class_prob in self.class_probs.items():
                posterior = class_prob
                print(f"  - Initial Posterior (P({class_value})): {posterior}")

                for column, value in row.items():
                    cond_prob = self.cond_probs[column].get((value, class_value), 0)
                    posterior *= cond_prob
                    print(f"    P({column} = {value} | {class_value}) = {cond_prob}")
                    print(f"    Updated Posterior (P({class_value} | X)): {posterior}")

                posteriors[class_value] = posterior
                print(f"  - Final Posterior for class {class_value}: {posterior}")
            
            total_posterior = sum(posteriors.values())
            print(f"  - Total Posterior: {total_posterior}")
            if total_posterior > 0:
                posteriors = {k: v / total_posterior for k, v in posteriors.items()}
            print(f"  - Normalized Posterior: {posteriors}")
            
            predictions.append(max(posteriors, key=posteriors.get))  # Choose the class with the highest posterior
            print(f"  - Predicted Class: {predictions[-1]}")
            print("=" * 40)

        return predictions


# Example dataset
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes']
}

# Convert the data to DataFrame
df = pd.DataFrame(data)

# Feature columns and target column
X = df.drop('PlayTennis', axis=1)
y = df['PlayTennis']

# Initialize and train the Naive Bayes classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X, y)

# Sample prediction: Predict whether tennis is played based on some new data
new_data = pd.DataFrame({
    'Outlook': ['Sunny'],
    'Temperature': ['Hot'],
    'Humidity': ['High'],
    'Wind': ['Weak']
})

predictions = nb_classifier.predict(new_data)
print("Final Prediction for new data:", predictions)


Prior Probabilities (P(class)):
{'Yes': 0.6, 'No': 0.4}

Conditional Probabilities (P(x | class)):
{'Outlook': {('Sunny', 'No'): 0.75, ('Sunny', 'Yes'): 0.16666666666666666, ('Overcast', 'No'): 0.0, ('Overcast', 'Yes'): 0.3333333333333333, ('Rain', 'No'): 0.25, ('Rain', 'Yes'): 0.5}, 'Temperature': {('Hot', 'No'): 0.5, ('Hot', 'Yes'): 0.16666666666666666, ('Mild', 'No'): 0.25, ('Mild', 'Yes'): 0.3333333333333333, ('Cool', 'No'): 0.25, ('Cool', 'Yes'): 0.5}, 'Humidity': {('High', 'No'): 0.75, ('High', 'Yes'): 0.3333333333333333, ('Normal', 'No'): 0.25, ('Normal', 'Yes'): 0.6666666666666666}, 'Wind': {('Weak', 'No'): 0.5, ('Weak', 'Yes'): 0.8333333333333334, ('Strong', 'No'): 0.5, ('Strong', 'Yes'): 0.16666666666666666}}

Predicting for row 0: {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'}
  - Initial Posterior (P(Yes)): 0.6
    P(Outlook = Sunny | Yes) = 0.16666666666666666
    Updated Posterior (P(Yes | X)): 0.09999999999999999
    P(Temperature = Hot | 